In [3]:
!pip install numpy scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.9/35.9 MB 43.1 MB/s eta 0:00:0000:0100:01


In [7]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import resample

# --- CONFIG ---
input_wav = "ath4-11.wav"   # your WAV file in the container
array_name = "testTone"      # name of the C array
target_fs = 16000          # target sample rate for ESP32 I2S

# --- READ WAV ---
fs_orig, data = wavfile.read(input_wav)

# Convert to mono if needed
if data.ndim > 1:
    data = data[:, 0]  # take left channel

# Convert to int16 if needed
if data.dtype != np.int16:
    if np.issubdtype(data.dtype, np.floating):
        data = (data * 32767).astype(np.int16)
    else:
        data = data.astype(np.int16)

# --- RESAMPLE to target_fs ---
duration_sec = len(data) / fs_orig
target_samples = int(target_fs * duration_sec)
data_resampled = resample(data, target_samples).astype(np.int16)

# --- FORMAT AS C ARRAY ---
c_array = ', '.join(map(str, data_resampled))
header = f"const int16_t {array_name}[{len(data_resampled)}] = {{\n{c_array}\n}};"

# --- SAVE TO FILE ---
output_file = f"{array_name}.h"
with open(output_file, "w") as f:
    f.write(header)

print(f"C array written to {output_file} ({len(data_resampled)} samples at {target_fs} Hz)")


C array written to testTone.h (72235 samples at 16000 Hz)
